# 파라메터 서칭
tree base 모델은 설정 가능한 파라메터의 조합에 따라 모델 예측력 차이가 큰 특징을 가지고 있습니다.  
특히, Xgboost 모델의 경우 파라메터 설정에 따른 모델 예측력 차이가 굉장히 크기에 꼭 파라메터 서칭을 진행해주셔야 합니다.  
간단한 문법을 통해 파라메터 서칭을 진행 해보겠습니다.

## 파이썬 기본 문법으로 파라메터 서칭

In [2]:
# 필요 모듈 import
from itertools import product
import pandas as pd

In [3]:
# 데이터셋 로딩
df = pd.read_csv('./data/boston.csv')

In [7]:
# 타겟 데이터 분할
y = df['y']
X = df.drop('y', axis=1)

In [8]:
# 학습 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
# 검증 데이터 분할
X_train2, X_val, y_train2, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [14]:
from sklearn.ensemble import RandomForestRegressor
from itertools import product
from sklearn.metrics import r2_score

In [21]:
# 파라메터 서칭
depth = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
est = [20, 30, 40, 50, 60, 70, 80, 90, 100]

best_r2 = -999999999 # 최적결과를 저장하는 변수
best_param = 0 # 최적파라메터를 저장하는 변수

for param in list(product(depth, est)):
    print(f'{param} 서칭중')
    model = RandomForestRegressor(random_state=42, max_depth=param[0], n_estimators=param[1])
    model.fit(X_train2, y_train2)
    pred = model.predict(X_val)
    r2 = r2_score(y_val, pred)
    if r2 > best_r2:
        print('스코어 갱신!!!')
        best_r2 = r2
        best_param = param

(12, 20) 서칭중
스코어 갱신!!!
(12, 30) 서칭중
(12, 40) 서칭중
(12, 50) 서칭중
(12, 60) 서칭중
(12, 70) 서칭중
(12, 80) 서칭중
(12, 90) 서칭중
(12, 100) 서칭중
(13, 20) 서칭중
(13, 30) 서칭중
(13, 40) 서칭중
(13, 50) 서칭중
(13, 60) 서칭중
(13, 70) 서칭중
(13, 80) 서칭중
(13, 90) 서칭중
(13, 100) 서칭중
(14, 20) 서칭중
스코어 갱신!!!
(14, 30) 서칭중
(14, 40) 서칭중
(14, 50) 서칭중
(14, 60) 서칭중
(14, 70) 서칭중
(14, 80) 서칭중
(14, 90) 서칭중
(14, 100) 서칭중
(15, 20) 서칭중
스코어 갱신!!!
(15, 30) 서칭중
(15, 40) 서칭중
(15, 50) 서칭중
(15, 60) 서칭중
(15, 70) 서칭중
(15, 80) 서칭중
(15, 90) 서칭중
(15, 100) 서칭중
(16, 20) 서칭중
(16, 30) 서칭중
(16, 40) 서칭중
(16, 50) 서칭중
(16, 60) 서칭중
(16, 70) 서칭중
(16, 80) 서칭중
(16, 90) 서칭중
(16, 100) 서칭중
(17, 20) 서칭중
(17, 30) 서칭중
(17, 40) 서칭중
(17, 50) 서칭중
(17, 60) 서칭중
(17, 70) 서칭중
(17, 80) 서칭중
(17, 90) 서칭중
(17, 100) 서칭중
(18, 20) 서칭중
스코어 갱신!!!
(18, 30) 서칭중
(18, 40) 서칭중
(18, 50) 서칭중
(18, 60) 서칭중
(18, 70) 서칭중
(18, 80) 서칭중
(18, 90) 서칭중
(18, 100) 서칭중
(19, 20) 서칭중
(19, 30) 서칭중
(19, 40) 서칭중
(19, 50) 서칭중
(19, 60) 서칭중
(19, 70) 서칭중
(19, 80) 서칭중
(19, 90) 서칭중
(19, 100) 서칭중
(20, 20) 서칭중
(20

In [23]:
# 위 파라메터 서칭 코드로 찾은 최적 모델의 평가지표와 파라메터 확인
best_r2, best_param

(0.6534021977867805, (18, 20))

In [24]:
# 최적 모델로 모델 다시 학습 및 평가
best_model = RandomForestRegressor(random_state=42, max_depth=best_param[0], n_estimators=best_param[1])
best_model.fit(X_train, y_train)
best_pred = best_model.predict(X_test)
print(r2_score(y_test, best_pred))

0.8763025355694289


In [ ]:
'''
0.8708997131330258
3.1015579977098433
'''

## sklearn GridSearchCV
sklearn 패키지에는 위의 파라메터 서칭 과정을 간편하게 진행 할 수 있도록 GridSearchCV 방법론을 제공합니다.  
기존 파라메터 서칭과 함께 cross validation 과정을 추가하여 데이터 분할에 강건한 모델을 선택할 수 있도록 제작 되었습니다.

In [ ]:
''' xgboost 파라메터 서칭 목록
max_depth : classification [3, 5, 7, 9], regression [7, 9, 11, 13, 15]
n_estimators : 부스팅 트리 갯수, [100, 300, 500, 700, 1000, 2000, 3000]
learning_rate : round별 학습률 [0.001, 0.003, 0.01, 0.03]
subsample : 부트스트랩 샘플 비율 [0.6, 0.7, 0.8]
colsample_bytree : 부트스트랩 컬럼 비율 [0.6, 0.7, 0.8]
설명변수 갯수가 많은 경우 아래 파라메터도 서칭
reg_alpha : L1, lasso [1, 3, 5, 7, 9]
reg_lambda : L2, ridge, [1, 3, 5, 7, 9]
'''

In [ ]:
# 그리드 서치 import


In [ ]:
# 그리드 서치를 위한 모델 및 파라메터 준비


In [ ]:
# 그리드 서치 모델 생성

'''
estimator : 모델 딕셔너리
param_grid : 파라메터 딕셔너리
scoring=None : 평가방법
n_jobs=None : 학습에 사용할 컴퓨터 코어 갯수
verbose=0 : 리포트 형식 0, 1, 2

scoring 참고
https://scikit-learn.org/stable/modules/model_evaluation.html
'''

In [ ]:
# grid 학습


In [ ]:
# 최적 모델 및 파라메터 확인


In [ ]:
# 최적 모델로 모델 다시 학습 및 평가
